In [ ]:
! /opt/bin/nvidia-smi

Sun Dec  6 09:48:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content') 

Mounted at /content/drive


In [ ]:
import shutil
DIR = '/content/drive/MyDrive/Colab Notebooks/HW3'
shutil.copyfile(os.path.join(DIR, 'HW3.zip'), 'HW3.zip')
!unzip HW3.zip

Archive:  HW3.zip
   creating: HW3/
  inflating: HW3/.DS_Store           
   creating: __MACOSX/
   creating: __MACOSX/HW3/
  inflating: __MACOSX/HW3/._.DS_Store  
   creating: HW3/.ipynb_checkpoints/
  inflating: HW3/.ipynb_checkpoints/HW3-checkpoint.ipynb  
   creating: HW3/__pycache__/
  inflating: HW3/__pycache__/utils.cpython-37.pyc  
   creating: HW3/dataset/
  inflating: HW3/dataset/.DS_Store   
   creating: __MACOSX/HW3/dataset/
  inflating: __MACOSX/HW3/dataset/._.DS_Store  
   creating: HW3/dataset/test/
  inflating: HW3/dataset/test/2007_000629.jpg  
   creating: __MACOSX/HW3/dataset/test/
  inflating: __MACOSX/HW3/dataset/test/._2007_000629.jpg  
  inflating: HW3/dataset/test/2007_001175.jpg  
  inflating: __MACOSX/HW3/dataset/test/._2007_001175.jpg  
  inflating: HW3/dataset/test/2007_001239.jpg  
  inflating: __MACOSX/HW3/dataset/test/._2007_001239.jpg  
  inflating: HW3/dataset/test/2007_001733.jpg  
  inflating: __MACOSX/HW3/dataset/test/._2007_001733.jpg  
  inflating:

In [ ]:
os.chdir('/content/HW3')

from PixelLib.pixellib.custom_train import instance_custom_training
from utils import get_imagenet_weights
import glob

class_names = ['BG', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
save_models_path = os.path.join(DIR, 'mask_rcnn_models512')

def get_bestModel_weights():
  return os.path.join(os.path.abspath(save_models_path), sorted(os.listdir(save_models_path), reverse=True)[0])
   
#params
init_lr = 0.001
lr_steps = 5
max_dim = 512
min_dim = 512


In [ ]:
#infer
def infer():
  train_maskrcnn = instance_custom_training()
  train_maskrcnn.load_dataset("dataset")
  train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=1, detection_threshold=0.5, image_max_dim=max_dim, image_min_dim=min_dim)
  #train_maskrcnn.evaluate_model(model_path=save_models_path)
  train_maskrcnn.evaluate_model(model_path=get_bestModel_weights())


In [ ]:
infer()

Using resnet101 as network backbone For Mask R-CNN model
Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
/content/drive/MyDrive/Colab Notebooks/HW3/mask_rcnn_models512/mask_rcnn_model.018-2.979291.h5 evaluation using iou_threshold 0.5 is 0.160246 



In [ ]:
#train
train_maskrcnn = instance_custom_training()
train_maskrcnn.load_dataset("dataset")


#1st
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.7, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_imagenet_weights())
train_maskrcnn.train_model(num_epochs=10, augmentation=True, layers='heads', path_trained_models=save_models_path)
infer()
#2nd
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.7, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_bestModel_weights())
train_maskrcnn.train_model(num_epochs=20, augmentation=True, layers='4+', path_trained_models=save_models_path)
infer()
#3st
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.7, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_bestModel_weights())
train_maskrcnn.train_model(num_epochs=60, augmentation=True, layers='all', path_trained_models=save_models_path)
infer()
'''
#3st
train_maskrcnn.modelConfig(network_backbone="resnet101", num_classes=20, class_names=class_names, batch_size=8, detection_threshold=0.5, image_max_dim=max_dim, image_min_dim=min_dim, learning_rate=init_lr, lr_steps=lr_steps)
train_maskrcnn.load_pretrained_model(get_imagenet_weights())
train_maskrcnn.train_model(num_epochs=30, augmentation=True, layers='all', path_trained_models=save_models_path)
'''

Using resnet101 as network backbone For Mask R-CNN model
94658560/94653016 [==============================] - 3s 0us/step
Train 1349 images
Validate 1349 images
Applying augmentation on dataset
Checkpoint Path: /content/drive/MyDrive/Colab Notebooks/HW3/mask_rcnn_models512
Selecting layers to train
Learning Rate 0.005
Learning Rate 0.005
Epoch 1/3
 73/180 [===========>..................] - ETA: 5:18 - batch: 36.0000 - size: 8.0000 - loss: 7.6502 - rpn_class_loss: 0.9888 - rpn_bbox_loss: 4.5096 - mrcnn_class_loss: 0.4114 - mrcnn_bbox_loss: 1.0487 - mrcnn_mask_loss: 0.6917

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
from itertools import groupby
from pycocotools import mask as maskutil

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    compressed_rle = maskutil.frPyObjects(rle, rle.get('size')[0], rle.get('size')[1])
    compressed_rle['counts'] = str(compressed_rle['counts'], encoding='utf-8')
    return compressed_rle

In [ ]:
from PixelLib.pixellib.instance import custom_segmentation 
from utils import binary_mask_to_rle
import json, os
import numpy as np

json_path = '/content/HW3/dataset/test.json'
with open(json_path) as f:
   test_json = json.load(f)

test_path = '/content/HW3/dataset/test'
save_dir = os.path.join(DIR, 'test_predict')
if not os.path.exists(save_dir):
  os.mkdir(save_dir)

submissions = []

segment_image = custom_segmentation()
segment_image.inferConfig(num_classes=20, class_names=class_names, image_max_dim=max_dim, image_min_dim=min_dim, detection_threshold=0.5)
segment_image.load_model(get_bestModel_weights())

for t in test_json['images']:
  filename = t['file_name']
  id = t['id']
  filepath = os.path.join(test_path, filename)
  save_path = os.path.join(save_dir, filename)
  results, o = segment_image.segmentImage(filepath, show_bboxes=True, output_image_name=save_path)
  #print(results)
  category_ids = results['class_ids']
  scores = results['scores']
  segmentations = results['masks']
  n_instances = len(scores)    
  if len(category_ids) > 0: # If any objects are detected in this image
    for i in range(n_instances):
      instance = dict()
      category_id, score, segmentation = category_ids[i], scores[i], binary_mask_to_rle(segmentations[:,:,i])
      #print(segmentations[:,:,i])
      instance['image_id'], instance['category_id'], instance['segmentation'], instance['score'] = \
        int(id), int(category_id), segmentation, float(score)
      submissions.append(instance)
      #print(segmentation)

#submissions = tuple(submissions)
with open(os.path.join(DIR,"submission.json"), "w") as f:
    json.dump(submissions, f)

Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image saved successfully in your current working directory.
Processed Image save